In [ ]:
# Importar las bibliotecas
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from openai import OpenAI
import faiss
import numpy as np

Mejor tener el modelo en embedings descagado de HugginFaces que tarda mucho en cargar (lede salesforce son 15 gigas)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Cargar el tokenizer y el modelo
model_name = "BAAI/llm-embedder"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
# Cargar y dividir el documento
def load_and_split_document(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    return texts

# Generar embeddings
def generate_embeddings(texts):
    embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

    #embed_model = HuggingFaceEmbeddings(model_name="Salesforce/SFR-Embedding-2_R")
    #embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v2")
    #embed_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
    embeddings = embed_model.embed_documents([text.page_content for text in texts])

    return embeddings, embed_model

# Crear un índice FAISS
def create_faiss_index(embeddings):
    embeddings_array = np.array(embeddings)
    dimension = embeddings_array.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_array)
    return index

# Recuperar el contexto relevante
def retrieve_context(query, index, texts, embed_model, top_k=4):
    query_embedding = embed_model.embed_query(query)
    query_embedding = np.array(query_embedding).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    relevant_chunks = [texts[i].page_content for i in indices[0]]
    return relevant_chunks



embed_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

In [ ]:
# Preguntar a DeepSeek
def ask_deepseek(query, index, texts, embed_model):
    client = OpenAI(api_key="", base_url="https://api.deepseek.com")

    # Recuperar el contexto relevante
    context = retrieve_context(query, index, texts, embed_model)
    context_str = "\n".join(context)

    # Crear el mensaje para DeepSeek
    messages = [
       # {"role": "system", "content": "Eres atención a cliente de un colegio da respuestas escuetas y concisas basadas en la información del docuemnto adjunto responde en el mismo idioma que te pregunten"},
        {"role": "system", "content": "Eres un inspector de educación responde docuemnto adjunto responde en el mismo idioma que te pregunten y opina sobre la cuestión como inspector"},
        {"role": "user", "content": f"Context:\n{context_str}\n\nQuestion: {query}"},
    ]

    # Enviar la solicitud a DeepSeek
    response = client.chat.completions.create(
      #  model="deepseek-chat",
      model="deepseek-chat",
        messages=messages,
        stream=False
    )

    return response.choices[0].message.content

# Cargar el documento y dividirlo en chunks
file_path = "/content/documento.txt"  # Cambia la ruta a tu archivo
texts = load_and_split_document(file_path)

# Generar embeddings
embeddings, embed_model = generate_embeddings(texts)

# Crear el índice FAISS
index = create_faiss_index(embeddings)

# Ejemplo de uso
query = "promoción de primero a segundo"
response = ask_deepseek(query, index, texts, embed_model)
print("Respuesta de DeepSeek:\n", response)

In [ ]:
# Ejemplo de uso
query = "estudia y da tu opinion sobre principios y valores "
response = ask_deepseek(query, index, texts, embed_model)
print("Respuesta de DeepSeek:\n", response)

In [ ]:
# Ejemplo de uso
query = "analysiert und gibt eine Stellungnahme zum KOEXISTENZBEREICH ab"
response = ask_deepseek(query, index, texts, embed_model)
print("Respuesta de DeepSeek:\n", response)

Ámbito de Convivencia (Koexistenzbereich) se refiere al clima social, la gestión de conflictos y la promoción de una convivencia respetuosa en la comunidad escolar. Del contexto de la Metodología 8.1.4. del Plan de Acción Anual de los Tutores se desprenden los siguientes aspectos clave:

Integración en la autoevaluación y mejora:
El plan enfatiza que el diagnóstico debe abarcar todas las áreas de la escuela, incluyendo los seis factores clave (presumiblemente también el ámbito de convivencia). Esto es positivo, ya que una convivencia armoniosa es un requisito para un aprendizaje efectivo. Sin embargo, faltan herramientas o indicadores concretos para identificar conflictos, acoso escolar o la efectividad de las medidas preventivas.

Rol de las TIC para fomentar la diversidad:
El énfasis en las TIC como herramienta para garantizar la diversidad es relevante, por ejemplo, mediante plataformas digitales para la comunicación inclusiva o formación en ética digital. El ámbito de convivencia podría beneficiarse más si se utilizaran las TIC de manera específica para fomentar la empatía (por ejemplo, proyectos de intercambio virtual) o para la denuncia anónima de conflictos.

Evaluación y mejora continua:
El plan establece que las necesidades de desarrollo se derivan de la autoevaluación anual. Para el ámbito de convivencia, esto implica que deberían realizarse encuestas regulares entre estudiantes, profesores y padres para identificar fortalezas y debilidades. No obstante, no queda claro cómo se traducen estos datos en acciones concretas (por ejemplo, programas de mediación, talleres de inteligencia emocional).

Evaluación crítica como inspector escolar:

Fortalezas: La integración del ámbito de convivencia en el ciclo de mejora escolar está bien planteada en términos generales. El énfasis en las TIC como elemento de apoyo muestra modernidad.

Debilidades: Faltan estrategias específicas para la prevención e intervención en conflictos. El diagnóstico es demasiado general, sin criterios claros o indicadores cuantitativos/cualitativos para el ámbito de convivencia.

Recomendaciones:

Desarrollo de un catálogo de medidas concretas para el ámbito de convivencia (por ejemplo, mediación entre iguales, gestión digital de quejas).

Involucrar a expertos externos (por ejemplo, psicólogos escolares) en la autoevaluación.

Utilizar herramientas TIC para la recopilación anónima de conflictos y fomentar la participación (por ejemplo, aplicaciones de feedback digital).

Formación regular para el profesorado en "Aprendizaje Social y Ciudadanía Digital".

